# **Before each experiment**

Select `Kernel` > `Restart Kernel and Clear All Outputs`

In [ ]:
%%cmd
poetry update

***
# **Without mouse on stage**

In [1]:
%env AIBS_RIG_ID = NP.0
# for testing

env: AIBS_RIG_ID=NP.0


In [2]:
import np_workflows
import np_services
import np_workflows.npxc as npxc
import np_workflows.experiments.task_trained_network as TTN

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [3]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [ ]:
np_services.start_rsc_apps()

***
## Select mouse and user

In [4]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select session

In [6]:
np_workflows.mtrain_widget(mouse)
selected_session = TTN.stim_session_select_widget(mouse)

***
## Generate new session

In [4]:
experiment = TTN.new_experiment(mouse, user, selected_session)
platform_json = experiment.platform_json

***
## Pretest
### Checks before running

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
np_workflows.check_openephys_widget()

### Pretest critical components individually
Only proceed if all pretests pass!

In [ ]:
np_services.MouseDirector.pretest()

In [ ]:
np_services.OpenEphys.pretest()

In [ ]:
np_services.VideoMVR.pretest()

In [ ]:
np_services.Cam3d.pretest()

In [ ]:
np_services.Sync.pretest()

In [ ]:
np_services.NewScaleCoordinateRecorder.pretest()

In [ ]:
np_services.ScriptCamstim.pretest()

***
## Setup components for the experiment and reset all

In [ ]:
experiment.initialize_and_test_services()

***
## Dip probes

In [ ]:
np_workflows.di_widget(platform_json)
np_workflows.photodoc_widget('pre_experiment_surface_image')

***
***
# **With mouse on stage**

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()

In [ ]:
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## Probe insertion

In [ ]:
platform_json.ProbeInsertionStartTime = npxc.now()
np_workflows.isi_widget(mouse.lims, colormap=False)

***
## Photodoc before advancing probes

In [ ]:
np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer

In [ ]:
np_workflows.print_countdown_timer(minutes=1)

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Start experiment

In [19]:
experiment.start_stim()

21:27 | ttn_pilot | WARNING | Using hard-coded script in notebooks directory for testing
21:27 | root | INFO | Starting Camstim script
21:27 | np_services.proxies | INFO | Finalizing ScriptCamstim
21:27 | np_services.proxies | INFO | ScriptCamstim added new data: []
21:27 | root | INFO | Camstim script complete
21:27 | root | INFO | Starting opto-tagging script
21:33 | np_services.proxies | INFO | Finalizing ScriptCamstim
21:33 | np_services.proxies | INFO | ScriptCamstim added new data: ['230216212712_366122.opto.pkl']
21:33 | root | INFO | Opto-tagging script complete


***
## Stop recording

In [ ]:
experiment.stop_services()

***
## Before removing probes

In [ ]:
np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()
platform_json.write()

np_workflows.finishing_checks_widget()

In [ ]:
experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)
experiment.copy_files()